In [158]:
import numpy as np
with open("rosalind_ba5k.txt") as f:
    strings = [line.rstrip() for line in f.readlines()]

s,t = strings[0], strings[1]



In [159]:
blosum62 = [    
    [4, 0, -2, -1, -2, 0, -2, -1, -1, -1, -1, -2, -1, -1, -1, 1, 0, 0, -3, -2],
    [0, 9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2],
    [-2, -3, 6, 2, -3, -1, -1, -3, -1, -4, -3, 1, -1, 0, -2, 0, -1, -3, -4, -3],
    [-1, -4, 2, 5, -3, -2, 0, -3, 1, -3, -2, 0, -1, 2, 0, 0, -1, -2, -3, -2],
    [-2, -2, -3, -3, 6, -3, -1, 0, -3, 0, 0, -3, -4, -3, -3, -2, -2, -1, 1, 3],
    [0, -3, -1, -2, -3, 6, -2, -4, -2, -4, -3, 0, -2, -2, -2, 0, -2, -3, -2, -3],
    [-2, -3, -1, 0, -1, -2, 8, -3, -1, -3, -2, 1, -2, 0, 0, -1, -2, -3, -2, 2],
    [-1, -1, -3, -3, 0, -4, -3, 4, -3, 2, 1, -3, -3, -3, -3, -2, -1, 3, -3, -1],
    [-1, -3, -1, 1, -3, -2, -1, -3, 5, -2, -1, 0, -1, 1, 2, 0, -1, -2, -3, -2],
    [-1, -1, -4, -3, 0, -4, -3, 2, -2, 4, 2, -3, -3, -2, -2, -2, -1, 1, -2, -1],
    [-1, -1, -3, -2, 0, -3, -2, 1, -1, 2, 5, -2, -2, 0, -1, -1, -1, 1, -1, -1],
    [-2, -3, 1, 0, -3, 0, 1, -3, 0, -3, -2, 6, -2, 0, 0, 1, 0, -3, -4, -2],
    [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2, 7, -1, -2, -1, -1, -2, -4, -3],
    [-1, -3, 0, 2, -3, -2, 0, -3, 1, -2, 0, 0, -1, 5, 1, 0, -1, -2, -2, -1],
    [-1, -3, -2,  0, -3, -2, 0, -3, 2, -2, -1,  0, -2, 1, 5, -1, -1, -3, -3, -2],
    [1, -1, 0, 0, -2, 0, -1, -2, 0, -2, -1, 1, -1, 0, -1, 4, 1, -2, -3, -2], 
    [0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1, 0, -1, -1, -1, 1, 5, 0, -2, -2],
    [0, -1, -3, -2, -1, -3, -3, 3, -2, 1, 1, -3, -2, -2, -3, -2, 0, 4, -3, -1],
    [-3, -2, -4, -3, 1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11, 2],
    [-2, -2, -3, -2,  3, -3, 2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1,  2, 7]]
blosum62_aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
blosum62_idx = dict(zip(blosum62_aa, range(len(blosum62_aa))))


In [160]:
def best_score(s, t, col, cost):
    #initiate prev column
    prev = [-(i * cost) for i in range(len(s)+1)]
    for j in range(1,col+1):
        #initiate curr column
        curr = [0 for k in range(len(s)+1)]
        #first value comes from gap
        curr[0] = - j * cost
        #for each row 
        for i in range(1,len(s)+1):
            scores = [prev[i-1] + blosum62[blosum62_idx[s[i-1]]][blosum62_idx[t[j-1]]],
                    curr[i-1]  - cost,
                    prev[i]   - cost]

            best = np.argmax(scores)
            curr[i] = scores[best]

        #update prev and curr
        prev = curr
        # print(curr)
    return curr




In [161]:
middle_column = len(t)//2
#find best score for the first half of the matrix
f = best_score(s, t, middle_column, 5) #f[i] is the edit distance of t[0,middle_column] to s[0,i-1]
#find best score for the second half of the matrix
g = best_score(s[::-1],t[::-1],len(t) - middle_column, 5)[::-1] #g[i] is the edit distance of t[middle_column+1, len(t)] to s[i, len(s)]
## note g column's 0th element is next to f columns 1st element, because of the position of placeholder gap row on the reversed half of the matrix

In [162]:
#scores of edges of transition between first and second half of the matrix
# f_e_g[i,k], where k is 0 or 1 is the length of the shortest path that goes through edge i,k; 
f_e_g = {}


In [163]:
import math

In [164]:
for i in range(0,len(s)+1):
    # 1 for indel, horizontal move, e.g (1,1) -> (1,2)
    # 0 for match/mismatch, diagonal move, e.g. (1,1) -> (2,2)
    # 
    if i < len(s): # check for i to not be the last cell of the column, so a diagonal move is possible
        f_e_g[i,0] = f[i] + g[i] + blosum62[blosum62_idx[s[i]]][blosum62_idx[t[middle_column]]]
    else:
        f_e_g[i,0] = -math.inf
    
    if i != 0:  # check for i to not be the first cell of the column, so a horizontal move is possible
         f_e_g[i,1] = f[i] + g[i-1] - 5
    else:
         f_e_g[i,1] = -math.inf



In [165]:
max_ = max(f_e_g, key=f_e_g.get)
from_i = max_[0]
from_j = middle_column
if max_[1] == 0:
    to_i = from_i+1
    to_j = from_j+1
elif max_[1] == 1:
    to_i = from_i
    to_j = from_j+1
elif max_[1] == 2:
    to_i = from_i+1
    to_j = from_j

print((from_i, from_j),(to_i , to_j))


(502, 523) (503, 524)
